# モジュールをインポート

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# モデルを定義

In [3]:
dropout_prob = 0.5
n_labels = 5
training_epochs = 10
batch_size = 300
learning_rate = 1e-4

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Conv2D, Conv3D
from tensorflow.keras.layers import Reshape, Flatten, Softmax
from tensorflow.keras.optimizers import Adam

def model(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    X_input = Input(shape = input_shape)

    conv_1 = Conv3D(filters=32, kernel_size=(3, 3, 3), padding="same", strides=(1, 1, 1), activation="elu")(X_input)
    conv_2 = Conv3D(filters=64, kernel_size=(3, 3, 3), padding="same", strides=(1, 1, 1), activation="elu")(conv_1)
    conv_3 = Conv3D(filters=128, kernel_size=(3, 3, 3), padding="same", strides=(1, 1, 1), activation="elu")(conv_2)
    shape = conv_3.get_shape().as_list()
    
    pool_2_flat = Reshape([shape[1], shape[2]*shape[3]*shape[4]])(conv_3)
    fc = Dense(1024, activation="elu")(pool_2_flat)
    fc_drop = Dropout(dropout_prob)(fc)
    
    lstm_in = Reshape([10, 1024])(fc_drop)
    lstm_1 = LSTM(units=1024, return_sequences=True, unit_forget_bias=True, dropout=dropout_prob)(lstm_in)
    rnn_output = LSTM(units=1024, return_sequences=False, unit_forget_bias=True)(lstm_1)
    
    shape_rnn_out = rnn_output.get_shape().as_list()
    fc_out = Dense(shape_rnn_out[1], activation="elu")(rnn_output)
    fc_drop = Dropout(dropout_prob)(fc_out)
    y_ = Dense(n_labels)(fc_drop)
    y_posi = Softmax()(y_)
#     X = Argmax()(X)
#     X = argmax()(X)
#     X = Dense(5, activation="elu")(X)
    
    
    model = Model(inputs = X_input, outputs = y_posi)
    return model

In [5]:
model = model(input_shape = (10, 10, 11, 1))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


# モデルの確認

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 10, 11, 1)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 10, 11, 32)    896       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 10, 11, 64)    55360     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 10, 11, 128)   221312    
_________________________________________________________________
reshape (Reshape)            (None, 10, 14080)         0         
_________________________________________________________________
dense (Dense)                (None, 10, 1024)          14418944  
_________________________________________________________________
dropout (Dropout)            (None, 10, 1024)          0     

In [7]:
opt = Adam(lr=learning_rate)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## 学習済みモデルの重みを読み込み

In [10]:
checkpoint_path = "./results/keras_2019_11_21/keras_model/cp.ckpt"
model.load_weights(checkpoint_path)

# Load data stream

In [11]:
from pylsl import StreamInlet, resolve_byprop
from muselsl.constants import LSL_SCAN_TIMEOUT, LSL_EEG_CHUNK

LSL_EEG_CHUNK = 10
def record_stream():
    """
    :return: a generator to fetch EEG data from existed stream.
    """
    streams = resolve_byprop("type", "EEG", timeout=LSL_SCAN_TIMEOUT)
    if len(streams) == 0:
        raise IOError("Can't find EEG stream.")

    inlet = StreamInlet(streams[0], max_buflen=LSL_EEG_CHUNK)

    info = inlet.info()
    description = info.desc()
    Nchan = info.channel_count()

    ch = description.child('channels').first_child()
    ch_names = [ch.child_value("label")]
    for i in range(1, Nchan):
        ch = ch.next_sibling()
        ch_names.append(ch.child_value('label'))
#     print(ch_names)
    while True:
        yield inlet.pull_chunk(timeout=1.0, max_samples=LSL_EEG_CHUNK)


In [12]:
print("['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX']")
print(np.array(next(record_stream())[0])[:, 0:4])

['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX']
[[  914.0625     -1000.          -890.13671875  -516.6015625 ]
 [ -489.74609375   136.71875     -424.8046875    971.19140625]
 [-1000.          -711.42578125  -146.97265625  -985.3515625 ]
 [ -262.20703125   474.12109375  -250.           -54.6875    ]
 [ -865.72265625 -1000.          -718.75        -932.6171875 ]
 [  119.62890625  -246.58203125  -565.4296875    425.78125   ]
 [-1000.          -862.79296875   -53.22265625  -830.078125  ]
 [ -811.5234375    994.62890625   -49.8046875    211.9140625 ]
 [  770.99609375  -543.9453125   -481.93359375 -1000.        ]
 [  329.58984375  -407.71484375  -681.15234375  -140.13671875]]


# Preprocessing

In [13]:
def data_1Dto2D(data, Y=10, X=11):
    data_2D = np.zeros([Y, X])
    data_2D[1, 3] = data[1]
    data_2D[1, 7] = data[2]
    data_2D[5, 1] = data[0]
    data_2D[5, 9] = data[3]
# 	data_2D[0] = ( 	   	 0, 	   0,  	   	 0, 	   0,        0,        0,        0, 	   0,  	     0, 	   0, 	 	 0) 
# 	data_2D[1] = (	  	 0, 	   0,  	   	 0, data[1],        0,        0,        0, data[2], 	   	 0,   	   0, 	 	 0) 
# 	data_2D[2] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 	 	 0) 
# 	data_2D[3] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[4] = (       0,        0,        0,        0,        0,        0,        0,        0,        0,        0,        0) 
# 	data_2D[5] = (	  	 0, data[0],        0,        0,        0,        0,        0,        0,        0, data[3], 		 0) 
# 	data_2D[6] = (	  	 0,        0,        0,        0,        0,        0,        0,        0,        0,        0, 		 0) 
# 	data_2D[7] = (	  	 0, 	   0, 	 	 0,        0,        0,        0,        0,        0, 	   	 0, 	   0, 		 0) 
# 	data_2D[8] = (	  	 0, 	   0, 	 	 0, 	   0,        0,        0,        0, 	   0, 	   	 0, 	   0, 		 0) 
# 	data_2D[9] = (	  	 0, 	   0, 	 	 0, 	   0, 	     0,        0, 		 0, 	   0, 	   	 0, 	   0, 		 0) 
    return data_2D

In [15]:
buffer_2d = np.zeros((10, 10, 11))

In [16]:
X_test = buffer_2d.reshape((1, 10, 10, 11, 1))

In [17]:
true_labels = ['eye_close', 'image_open&close_both_feet',
       'image_open&close_both_fists', 'image_open&close_left_fist',
       'image_open&close_right_fist']
for stream in record_stream():
    buffer_2d = np.zeros((10, 10, 11))
#     print(stream[0])
    for i in next(record_stream())[0]:
        num = 0
        buffer_2d[num, 1, 3] = i[1]
        buffer_2d[num, 1, 7] = i[2]
        buffer_2d[num, 5, 1] = i[0]
        buffer_2d[num, 5, 9] = i[3]
    X_test = buffer_2d.reshape((1, 10, 10, 11, 1))
    y_pred = model.predict(X_test, batch_size=1, verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(true_labels[int(y_pred_bool)])

image_open&close_both_fists
eye_close
eye_close
eye_close
eye_close
eye_close
eye_close
eye_close
eye_close
eye_close
eye_close
image_open&close_both_fists


KeyboardInterrupt: 

# テストデータを分類

In [ ]:
y_pred = model.predict(X_test, batch_size=1, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

In [ ]:
print("サンプリング率:", 160)
print("1秒に処理できる信号数：", 990*10-6)

In [ ]:
y_pred_bool = one_hot_encoder(y_pred_bool)
y_pred_bool

# 精度と正解率の曲線

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.preprocessing import label_binarize

precision = dict()
recall = dict()
for i in range(5):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],
                                                        y_pred_bool[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()

# 分類結果

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_bool))

In [ ]:
print("検証正解率：", history["val_acc"][-1])

In [ ]:
from sklearn.metrics import accuracy_score
print("テスト正解率：", accuracy_score(y_test, y_pred_bool))